In [1]:
from torch.utils import data
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import os
from PIL import Image
from torchvision.transforms import ToTensor

In [2]:
args = {
    'batch_size':32,
    'epochs':20,
    'no_cuda':False,
    'seed':1,
    'log_interval':10
}

In [3]:
# root directory
root_dir = "/home/andreasabo/Documents/HNProject/"

# data directory on current machine: abhishekmoturu, andreasabo, denizjafari, navidkorhani
data_dir = "/home/navidkorhani/Documents/HNProject/all_label_img/"

# read target df
csv_path = os.path.join(root_dir, "all_splits_100000.csv")
data_df = pd.read_csv(csv_path, usecols=['image_ids', 'view_train'])

In [4]:
train_df = data_df[data_df.view_train != 0]
test_df = data_df[data_df.view_train == 0]

print("# of images in train = {}, # of images in test = {}".format(len(train_df), len(test_df)))
train_ids = []
test_ids = []

for ind, row in train_df.iterrows():
    train_ids.append(row['image_ids'])

for ind, row in test_df.iterrows():
    test_ids.append(row['image_ids'])

partition = {'train':train_ids, 'test':test_ids}

# of images in train = 67592, # of images in test = 4867


In [5]:
class Dataset(data.Dataset):
  'Characterizes a dataset for PyTorch'
  def __init__(self, list_IDs):
        'Initialization'
        self.list_IDs = list_IDs

  def __len__(self):
        'Denotes the total number of samples'
        return len(self.list_IDs)

  def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        ID = self.list_IDs[index]

        # Load data and get label
        img_path = data_dir + ID + '.jpg'
        image = Image.open(img_path)#.convert('L')
        image = ToTensor()(image)

        return image

In [6]:
args["cuda"]= not args['no_cuda'] and torch.cuda.is_available()

torch.manual_seed(args['seed'])

device = torch.device("cuda:1" if args['cuda'] else "cpu")

print("device is", device)

kwargs = {'num_workers': 1, 'pin_memory': True} if args['cuda'] else {}

# Parameters
params = {'batch_size': args['batch_size'],
          'shuffle': True,
          'num_workers': 6}

if args['cuda']:
    params['num_workers']=1
    params['pin_memory']=True


# Data Loader
training_set = Dataset(partition['train'])
train_loader = data.DataLoader(training_set, **params)

test_set = Dataset(partition['test'])
test_loader = data.DataLoader(test_set, **params)

device is cuda:1


In [7]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(65536, 400)
        self.fc21 = nn.Linear(400, 40)
        self.fc22 = nn.Linear(400, 40)
        self.fc3 = nn.Linear(40, 400)
        self.fc4 = nn.Linear(400, 65536)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 65536))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar


model = VAE().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)


# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 65536), reduction='sum')

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return BCE + KLD


def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, data in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % args['log_interval'] == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item() / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))


def test(epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, data in enumerate(test_loader):
            data = data.to(device)
            recon_batch, mu, logvar = model(data)
            test_loss += loss_function(recon_batch, data, mu, logvar).item()
            if i == 0:
                n = min(data.size(0), 8)
                comparison = torch.cat([data[:n],
                                      recon_batch.view(args['batch_size'], 1, 256, 256)[:n]])
                save_image(comparison.cpu(),
                         'results/reconstruction_' + str(epoch) + '.png', nrow=n)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [8]:
if __name__ == "__main__":
    for epoch in range(1, args['epochs'] + 1):
        train(epoch)
        test(epoch)
        with torch.no_grad():
            sample = torch.randn(64, 40).to(device)
            sample = model.decode(sample).cpu()
            save_image(sample.view(64, 1, 256, 256),
                       'results/sample_' + str(epoch) + '.png')

Train Epoch: 1 [0/67592 (0%)]	Loss: 137737.203125
Train Epoch: 1 [320/67592 (0%)]	Loss: 150652.484375
Train Epoch: 1 [640/67592 (1%)]	Loss: 148804.671875
Train Epoch: 1 [960/67592 (1%)]	Loss: 141267.562500
Train Epoch: 1 [1280/67592 (2%)]	Loss: 140122.484375
Train Epoch: 1 [1600/67592 (2%)]	Loss: 139597.234375
Train Epoch: 1 [1920/67592 (3%)]	Loss: 139256.187500
Train Epoch: 1 [2240/67592 (3%)]	Loss: 138825.515625
Train Epoch: 1 [2560/67592 (4%)]	Loss: 138438.078125
Train Epoch: 1 [2880/67592 (4%)]	Loss: 138172.640625
Train Epoch: 1 [3200/67592 (5%)]	Loss: 138006.765625
Train Epoch: 1 [3520/67592 (5%)]	Loss: 137259.765625
Train Epoch: 1 [3840/67592 (6%)]	Loss: 136758.468750
Train Epoch: 1 [4160/67592 (6%)]	Loss: 136526.171875
Train Epoch: 1 [4480/67592 (7%)]	Loss: 135477.875000
Train Epoch: 1 [4800/67592 (7%)]	Loss: 136075.015625
Train Epoch: 1 [5120/67592 (8%)]	Loss: 137377.562500
Train Epoch: 1 [5440/67592 (8%)]	Loss: 136989.390625
Train Epoch: 1 [5760/67592 (9%)]	Loss: 135155.765625

Train Epoch: 1 [48320/67592 (71%)]	Loss: 119899.171875
Train Epoch: 1 [48640/67592 (72%)]	Loss: 121582.929688
Train Epoch: 1 [48960/67592 (72%)]	Loss: 120475.125000
Train Epoch: 1 [49280/67592 (73%)]	Loss: 121033.851562
Train Epoch: 1 [49600/67592 (73%)]	Loss: 121039.320312
Train Epoch: 1 [49920/67592 (74%)]	Loss: 120612.570312
Train Epoch: 1 [50240/67592 (74%)]	Loss: 118576.679688
Train Epoch: 1 [50560/67592 (75%)]	Loss: 120556.484375
Train Epoch: 1 [50880/67592 (75%)]	Loss: 120784.062500
Train Epoch: 1 [51200/67592 (76%)]	Loss: 119478.265625
Train Epoch: 1 [51520/67592 (76%)]	Loss: 120571.476562
Train Epoch: 1 [51840/67592 (77%)]	Loss: 120285.046875
Train Epoch: 1 [52160/67592 (77%)]	Loss: 119740.406250
Train Epoch: 1 [52480/67592 (78%)]	Loss: 124912.945312
Train Epoch: 1 [52800/67592 (78%)]	Loss: 122765.960938
Train Epoch: 1 [53120/67592 (79%)]	Loss: 123973.937500
Train Epoch: 1 [53440/67592 (79%)]	Loss: 121704.820312
Train Epoch: 1 [53760/67592 (80%)]	Loss: 121727.492188
Train Epoc

OSError: [Errno 12] Cannot allocate memory

In [ ]:
len(x)